clean messy dataset by clustering high-cardinality text features (brand, type) using fuzzy string matching.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
!pip install thefuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.4 MB/s eta 0:00:00


In [ ]:
#relace this with your data path.
data = pd.read_csv("/content/drive/Shareddrives/MLNS/final project/vestiaire.csv")

In [ ]:
data.head()

,product_id,product_type,product_name,product_description,product_keywords,product_gender_target,product_category,product_season,product_condition,product_like_count,...,warehouse_name,seller_id,seller_username,usually_ships_within,seller_country,seller_products_sold,seller_num_products_listed,seller_community_rank,seller_num_followers,seller_pass_rate
0,43247626,Wool mini skirt,Wool mini skirt Miu Miu Grey size S Internatio...,Miu Miu – Pleated mini skirt Size: 36 (S) Wai...,Miu Miu Wool Skirts,Women,Women Clothing,Autumn / Winter,Never worn,34.0,...,Tourcoing,25775970,vitalii25775970,NaN,Germany,3.0,14.0,0.0,13.0,0.0
1,43247441,Jacket,Jacket Barbara Bui Navy size 42 FR in Cotton,For selling nice women's suit Barbara Bui size...,Barbara Bui Cotton Jackets,Women,Women Clothing,All seasons,Very good condition,1.0,...,Tourcoing,13698770,olivia13698770,NaN,Belgium,0.0,0.0,0.0,8.0,0.0
2,43246517,Wool coat,Wool coat Comme Des Garcons White size S Inter...,Magnificent boiled wool coat. I bought it in t...,Comme Des Garcons Wool Coats,Women,Women Clothing,Autumn / Winter,Very good condition,2.0,...,Tourcoing,6042365,cecilia6042365,1-2 days,Spain,58.0,69.0,0.0,62.0,96.0
3,43246507,Mini skirt,Mini skirt MSGM Black size 38 IT in Polyester,MSGM Skirt Black Printed Raw-Edge & Embroidere...,MSGM Polyester Skirts,Women,Women Clothing,All seasons,Very good condition,0.0,...,Brooklyn,13172949,gretchen13172949,1-2 days,United States,63.0,274.0,126346.0,131.0,96.0
4,43246417,Vegan leather trousers,Vegan leather trousers LVIR Black size 36 FR i...,LVIR black grained faux leather trousers size ...,LVIR Vegan leather Trousers,Women,Women Clothing,All seasons,Very good condition,1.0,...,Crawley,2578605,crunchykat,3-5 days,United Kingdom,19.0,14.0,102821.0,40.0,89.0


In [ ]:
data["product_category"].value_counts().sort_values(ascending=False)/len(data)

,count
product_category,
Men Clothing,0.214413
Women Clothing,0.213430
Women Accessories,0.162150
Women Shoes,0.159723
Men Shoes,0.139282
Men Accessories,0.109688


In [ ]:
clothing_and_accessories = [
    # Men's Clothes
    "suit", "shirt", "tie",

    # Women's Clothes
    "dress", "blouse", "skirt", "tanktop", "blazer",

    # Uni-Sex
    "coat", "jacket", "t-shirt", "trousers", "jeans", "shorts",
    "jumper", "pullover", "cardigan", "sweatshirt", "hoodie",
    "gloves", "mittens",

    # Baby Clothes
    "romper suit", "nappy",

    # Holiday/Leisure Clothes
    "swimming trunks", "bikini", "swimsuit", "swimming costume", "bathing costume",

    # Nightwear
    "nightdress", "nightie", "pyjamas", "dressing gown",

    # Underwear
    # Men's
    "boxers", "Y-fronts",
    # Women's (Lingerie)
    "bra", "pants", "knickers", "stockings", "tights", "corset",
    # Uni-Sex
    "vest", "leggings",

    # Footwear
    "shoes", "sandals", "boots", "wellingtons", "slippers", "socks", 'heels',

    # Headwear
    "beret", "bowler hat", "trilby", "knitted hat", "woolen hat", "bobble hat", "cap",

    # Accessories
    "belt", "handbag", "headscarf", "scarf", "glasses", "sunglasses", "wristwatch", "hanger",

    # Jewellery
    "bracelet", "necklace", "ring", "earring"
]

In [ ]:
from thefuzz import fuzz
from thefuzz import process
import pandas as pd
import re

def clean_word(word):
    # Keep only alphabetic characters
    return re.sub(r'[^a-zA-Z]', '', word)

def map_product_type(product_type, reference_list, threshold=80):
    words = set(product_type.lower().split())
    best_match = None
    best_score = 0

    for word in words:
        clean = clean_word(word)
        if not clean:  # skip empty strings
            continue
        match, score = process.extractOne(clean, reference_list, scorer=fuzz.partial_ratio)
        if score > best_score:
            best_match = match
            best_score = score

    return best_match if best_score >= threshold else "other"

data['product_type_cleaned'] = data['product_type'].apply(
    lambda x: map_product_type(x, clothing_and_accessories)
)

In [ ]:
broad_category_mapping = {
    "shirt": "top",
    "blouse": "top",
    "t-shirt": "top",
    "tanktop": "top",
    "jumper": "top",
    "pullover": "top",
    "cardigan": "top",
    "sweatshirt": "top",
    "hoodie": "top",

    "trousers": "bottom",
    "jeans": "bottom",
    "shorts": "bottom",
    "skirt": "bottom",
    "leggings": "bottom",

    "dress": "one-piece",
    "romper suit": "one-piece",
    "nightdress": "one-piece",
    "nightie": "one-piece",
    "dressing gown": "one-piece",

    "coat": "outerwear",
    "jacket": "outerwear",
    "suit": "outerwear",
    "blazer": "outerwear",

    "bikini": "swimwear",
    "swimming trunks": "swimwear",
    "swimsuit": "swimwear",
    "swimming costume": "swimwear",
    "bathing costume": "swimwear",

    "boxers": "underwear",
    "Y-fronts": "underwear",
    "pants": "underwear",
    "knickers": "underwear",
    "bra": "underwear",
    "vest": "underwear",
    "stockings": "underwear",
    "tights": "underwear",
    "pyjamas": "underwear",
    "nappy": "underwear",
    "corset": "underwear",

    "shoes": "footwear",
    "sandals": "footwear",
    "boots": "footwear",
    "wellingtons": "footwear",
    "slippers": "footwear",
    "socks": "footwear",
    "heels": "footwear",

    "beret": "accessory",
    "bowler hat": "accessory",
    "trilby": "accessory",
    "knitted hat": "accessory",
    "woolen hat": "accessory",
    "bobble hat": "accessory",
    "cap": "accessory",
    "belt": "accessory",
    "handbag": "accessory",
    "headscarf": "accessory",
    "scarf": "accessory",
    "glasses": "accessory",
    "sunglasses": "accessory",
    "wristwatch": "accessory",
    "hanger": "accessory",
    "bracelet": "accessory",
    "necklace": "accessory",
    "ring": "accessory",
    "earring": "accessory",
    "mittens": "accessory",
    "gloves": "accessory",
    "tie": "accessory",


    "other": "other"
}

In [ ]:
def map_to_broad_category(cleaned_type, category_map):
    return category_map.get(cleaned_type, "other")

data['broad_type'] = data['product_type_cleaned'].apply(
    lambda x: map_to_broad_category(x, broad_category_mapping)
)

In [ ]:
data["broad_type"].value_counts()/len(data)

,count
broad_type,
accessory,0.283555
other,0.208000
top,0.134785
footwear,0.126245
outerwear,0.087188
bottom,0.065138
one-piece,0.051205
underwear,0.043027
swimwear,0.000857


In [ ]:
high_fashion_brands = [
    # French luxury maisons
    "Chanel", "Dior", "Louis Vuitton", "Hermès", "Givenchy", "Balmain", "Celine", "Saint Laurent", "Lanvin", "Jean Paul Gaultier", "Chloé",

    # Italian powerhouses
    "Gucci", "Prada", "Miu Miu", "Valentino", "Versace", "Fendi", "Bottega Veneta", "Moschino", "Salvatore Ferragamo", "Etro", "Emporio Armani",

    # British icons
    "Burberry", "Alexander McQueen", "Stella McCartney", "Vivienne Westwood",

    # American fashion
    "Tom Ford", "Marc Jacobs", "Ralph Lauren", "Calvin Klein", "Michael Kors", "The Row", "Tory Burch",

    # Belgian & Japanese avant-garde
    "Dries Van Noten", "Ann Demeulemeester", "Maison Margiela", "Comme des Garçons", "Issey Miyake", "Yohji Yamamoto", "Sacai",

    # Korean/Japanese contemporary
    "Andersson Bell", "Wooyoungmi", "Ader Error", "Ambush", "AMI",

    # Luxury streetwear / contemporary crossover
    "Off-White", "Palm Angels", "Jacquemus", "Acne Studios", "Rick Owens", "Fear of God", "Alyx", "Vetements", "Balenciaga", "Marine Serre"

    # Jewelry and accessories
    "Cartier", "Tiffany & Co.", "Bvlgari", "Chopard", "Van Cleef & Arpels"
]

In [ ]:
def clean_brand_name(name):
    # Remove non-letter characters and lowercase
    return re.sub(r'[^a-zA-Z ]', '', name).strip().lower()

def match_high_fashion_brand(brand_name, brand_list, threshold=85):
    clean_name = clean_brand_name(brand_name)
    match, score = process.extractOne(clean_name, brand_list, scorer=fuzz.token_sort_ratio)
    if score >= threshold:
        return match
    else:
        return "other"

In [ ]:
data['brand_cleaned'] = data['brand_name'].apply(
    lambda x: match_high_fashion_brand(x, high_fashion_brands)
)

In [ ]:
data['brand_cleaned'].value_counts()/len(data)

,count
brand_cleaned,
other,0.674970
Gucci,0.045540
Burberry,0.026671
Prada,0.023289
Hermès,0.020778
Louis Vuitton,0.020736
Chanel,0.016668
Balenciaga,0.011739
Dior,0.011651


In [ ]:
material_mapping_refined = {
    # Direct single material groups
    "Cotton": "cotton",
    "Cotton - elasthane": "cotton",
    "Wool": "wool",
    "Linen": "linen",
    "Silk": "silk",
    "Cashmere": "cashmere",

    # Synthetic
    "Polyester": "synthetic",
    "Polyamide": "synthetic",
    "Viscose": "synthetic",
    "Lycra": "synthetic",
    "Spandex": "synthetic",
    "Synthetic": "synthetic",

    # Real Leather
    "Leather": "real_leather",
    "Suede": "real_leather",
    "Patent leather": "real_leather",
    "Exotic leathers": "real_leather",
    "Python": "real_leather",
    "Alligator": "real_leather",
    "Crocodile": "real_leather",
    "Ostrich": "real_leather",
    "Lizard": "real_leather",
    "Water snake": "real_leather",
    "Eel": "real_leather",
    "Pony-style calfskin": "real_leather",
    "Varan": "real_leather",

    # Fake Leather
    "Vegan leather": "fake_leather",

    # Fur
    "Fur": "fur",
    "Faux fur": "fur",
    "Shearling": "fur",
    "Mink": "fur",
    "Fox": "fur",
    "Beaver": "fur",
    "Rabbit": "fur",
    "Raccoon": "fur",
    "Chinchilla": "fur",
    "Mongolian Lamb": "fur",
    "Astrakhan": "fur",

    # Denim
    "Denim - Jeans": "denim",

    # Gem
    "Pearl": "gem",
    "Pearls": "gem",
    "Crystal": "gem",
    "Glass": "gem",

    # Gold
    "Gold": "gold",
    "Gold plated": "gold",
    "White gold": "gold",
    "Yellow gold": "gold",
    "Pink gold": "gold",
    "gold and steel": "gold",

    # Silver
    "Silver": "silver",
    "Silver Plated": "silver",
    "Silver Gilt": "silver",

    # Other
    "Lace": "other",
    "Glitter": "other",
    "Tweed": "other",
    "Cloth": "other",
    "Plastic": "other",
    "Rubber": "other",
    "Wood": "other",
    "Sponge": "other",
    "Wicker": "other",
    "Ceramic": "other",
    "Chain": "other",
    "Metal": "other",
    "Platinum": "other",
    "Titanium": "other",
    "Steel": "other",
    "Stingray": "other",
    "Not specified": "other",
    float('nan'): "other"
}

def map_refined_material(material):
    return material_mapping_refined.get(material, "other")

data['material_cleaned'] = data['product_material'].apply(map_refined_material)

In [ ]:
data["material_cleaned"].value_counts()/len(data)

,count
material_cleaned,
real_leather,0.301479
cotton,0.193300
other,0.165100
synthetic,0.132458
wool,0.078249
silk,0.074735
cashmere,0.016015
denim,0.010582
fur,0.008588


In [ ]:
feature_columns = ["product_type_cleaned", "broad_type","product_gender_target", "product_category",  "product_color",  "brand_cleaned","product_condition", "material_cleaned", "product_like_count"]
target_columns = ["price_usd"]

In [ ]:
data[feature_columns+target_columns][data["brand_cleaned"]!="other"].to_csv("/content/drive/Shareddrives/MLNS/final project/processed_data.csv")